# Preprocessing Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    

for f in file_path_list:


    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
     
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute('''CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = 
                {'class' : 'SimpleStrategy', 'replication_factor' : 1}''')

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [20]:
query1_table=""" CREATE TABLE IF NOT EXISTS song_artist_by_session 
(session_id int, session_item int, artist text, song_title text, song_lenght float, PRIMARY KEY (session_id, session_item))
"""
session.execute(query1_table)

'''
Creates the table for query1 with primary key on session_id and session_item

'''

'\nCreates the table for query1 with primary key on session_id and session_item\n\n'

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_artist_by_session (session_id, session_item,artist, song_title, song_lenght)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

'''
Reads the CSV-files and imports values into the songs table

'''
        

'\nReads the CSV-files and imports values into the songs table\n\n'

#### Do a SELECT to verify that the data have been inserted into each table

In [24]:
query1_select = """
SELECT artist, song_title, song_lenght, session_id, session_item FROM song_artist_by_session
WHERE session_id=338
AND session_item = 4
"""
rows = session.execute(query1_select)
for row in rows:
    print (row.artist, row.song_title, row.song_lenght, row.session_id, row.session_item)

'''
Selects every needed information (artist name, song title, lenght of the song) from the songs table.
Additionally the session ID and the item in session is provided for validation.
'''
    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


'\nSelects every needed information (artist name, song title, lenght of the song) from the songs table.\nAdditionally the session ID and the item in session is provided for validation.\n'

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [25]:
query2_table=""" CREATE TABLE IF NOT EXISTS artist_by_user_sorted 
(user_id int, session_id int, session_item int, artist text, song_title text, user_name_first text, user_name_last text, PRIMARY KEY((user_id, session_id), session_item))
"""
session.execute(query2_table)      
'''
Creates the table for query1 with primary key on user_id and session_id; additionally session_item is also provided int he PRIMARY KEY argument for sorting reasons.
'''

'\nCreates the table for query1 with primary key on user_id and session_id; additionally session_item is also provided int he PRIMARY KEY argument for sorting reasons.\n'

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_by_user_sorted  (user_id, session_id, session_item, artist, song_title, user_name_first, user_name_last)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),line[0], line[9], line[1], line[4]))
        

In [27]:
query2_select = """
SELECT artist, song_title, user_name_first, user_name_last, session_item FROM artist_by_user_sorted 
WHERE user_id=10
AND session_id = 182
"""
rows = session.execute(query2_select)
for row in rows:
    print (row.artist, row.song_title, row.user_name_first, row.user_name_last, row.session_item)
    
'''
Selects every needed information (artist name, song title, the first and last name of the user) from the artists table.
Additionally the information on item in session is provided for validation, that the output is sorted properly.
'''

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


'\nSelects every needed information (artist name, song title, the first and last name of the user) from the artists table.\nAdditionally the information on item in session is provided for validation, that the output is sorted properly.\n'

In [28]:
query3_table=""" CREATE TABLE IF NOT EXISTS user_filtered 
(song_title text, user_id int, user_name_last text, user_name_first text, PRIMARY KEY(song_title, user_id))
"""    
session.execute(query3_table)
'''
Creates the table for query3 with primary key on song_title;
additionally the last name and the first name of a user is provided in the PRIMARY KEY argument for sorting reasons (I just made that out of interest).
'''

'\nCreates the table for query3 with primary key on song_title;\nadditionally the last name and the first name of a user is provided in the PRIMARY KEY argument for sorting reasons (I just made that out of interest).\n'

In [17]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_filtered (song_title, user_id, user_name_first, user_name_last)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        

In [29]:
query3_select = """
SELECT song_title, user_name_first, user_name_last FROM user_filtered
WHERE song_title='All Hands Against His Own'
"""
rows = session.execute(query3_select)
for row in rows:
    print (row.user_name_first, row.user_name_last, row.song_title)

'''
Selects every needed information (the first and last name of the user) from the users table.
Additionally the song title is provided for validation, that the output is filtered properly.
'''

Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


'\nSelects every needed information (the first and last name of the user) from the users table.\nAdditionally the song title is provided for validation, that the output is filtered properly.\n'

### Drop the tables before closing out the sessions

In [31]:
session.execute("DROP TABLE IF EXISTS song_artist_by_session")
session.execute("DROP TABLE IF EXISTS artist_by_user_sorted")
session.execute("DROP TABLE IF EXISTS user_filtered")

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()